In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the Cleveland Heart Disease dataset
cleveland_data = pd.read_csv('Heart_disease_cleveland_new.csv', names=[
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
], na_values='?')

# Load the Statlog (Heart) dataset from local file
statlog_file_path = 'heart.dat'
statlog_column_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
]
statlog_data = pd.read_csv(statlog_file_path, names=statlog_column_names, delim_whitespace=True)

# Adjust 'slope' and 'thal' values in Statlog dataset to match Cleveland dataset
statlog_data['slope'] = statlog_data['slope'].replace({1: 1, 2: 2, 3: 0})
statlog_data['thal'] = statlog_data['thal'].replace({3: 1, 6: 2, 7: 3})
statlog_data['target'] = statlog_data['target'].replace({1: 0, 2: 1})

# Combine the datasets
data = pd.concat([cleveland_data, statlog_data])


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [ ]:
# Convert target column to numeric and handle missing values
data['target'] = pd.to_numeric(data['target'], errors='coerce')
data.dropna(inplace=True)
data['target'] = data['target'].astype(int)

# Split the data into features and target
X = data.drop('target', axis=1)
y = data['target'].apply(lambda x: 1 if x > 0 else 0)  # Convert target to binary

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
import random

class JellyfishOptimizationAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iter, b=0.5, alpha=1.5):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = np.array(lower_bound)
        self.upper_bound = np.array(upper_bound)
        self.max_iter = max_iter
        self.b = b  # Distribution coefficient
        self.alpha = alpha  # Alpha controls the movement step size
        self.population = self.initialize_population()
        self.best_solution = None
        self.best_fitness = -1

    def initialize_population(self):
        return np.random.uniform(self.lower_bound, self.upper_bound, (self.population_size, self.dimensions))

    def ocean_current(self, individual, mean_location):
        r = random.random()
        trend = self.best_solution - self.b * mean_location * r
        new_position = individual + r * trend
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def passive_motion(self, individual):
        new_position = individual + (self.upper_bound - self.lower_bound) * np.random.random(self.dimensions)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def active_motion(self, individual):
        r = random.random()
        new_position = individual + self.alpha * r * (self.best_solution - individual)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def optimize(self, fitness_function):
        self.best_solution = None
        self.best_fitness = -1

        for iteration in range(self.max_iter):
            c_t = iteration / self.max_iter
            mean_location = np.mean(self.population, axis=0)

            for i in range(self.population_size):
                individual = self.population[i]
                fitness_value = fitness_function(individual)
                if fitness_value > self.best_fitness:
                    self.best_fitness = fitness_value
                    self.best_solution = individual

            new_population = np.zeros_like(self.population)
            for i in range(self.population_size):
                if c_t < 0.5:
                    new_population[i] = self.ocean_current(self.population[i], mean_location)
                else:
                    if 1 - c_t < random.random():
                        new_population[i] = self.passive_motion(self.population[i])
                    else:
                        new_population[i] = self.active_motion(self.population[i])

            self.population = new_population

        return self.best_solution, self.best_fitness


In [ ]:
def create_ann_model(input_shape, layer_sizes, activation='relu', output_activation='sigmoid'):
    model = Sequential()
    model.add(tf.keras.Input(shape=(input_shape,)))
    for size in layer_sizes:
        model.add(Dense(size, activation=activation))
    model.add(Dense(1, activation=output_activation))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def ann_fitness(solution):
    layer_sizes = [int(size) for size in solution]
    model = create_ann_model(X_train.shape[1], layer_sizes)
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    _, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return accuracy


In [ ]:
# Define the bounds for the layer sizes of the ANN
lower_bound = [10, 10]  # Minimum number of neurons in each layer
upper_bound = [100, 100]  # Maximum number of neurons in each layer

# Instantiate the Jellyfish Optimization Algorithm
joa = JellyfishOptimizationAlgorithm(
    population_size=30, dimensions=2, lower_bound=lower_bound, upper_bound=upper_bound, max_iter=50
)

# Optimize the ANN parameters using JOA
best_solution, best_fitness = joa.optimize(ann_fitness)
print(f"Best solution (layer sizes): {best_solution}, Best fitness: {best_fitness}")

# Train the ANN model with the best parameters
layer_sizes = [int(size) for size in best_solution]
best_model = create_ann_model(X_train.shape[1], layer_sizes)
best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate the model
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Append the results to the CSV file
results = pd.DataFrame({
    'Model': ['Neural Network'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-Score': [f1]
})

results.to_csv('model_results.csv', mode='a', header=False, index=False)

Best solution (layer sizes): [100. 100.], Best fitness: 0.9213973879814148
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Accuracy: 0.8434782608695652
Precision: 0.8333333333333334
Recall: 0.8
F1 Score: 0.8163265306122449
ROC AUC Score: 0.8384615384615385
